In [1]:
import sympy as sp
import numpy as np
from sympy import Matrix, zeros, init_printing, symbols, diff
from IPython.display import Image
#display(Image("q4.png"))
init_printing(pretty_print = True)
X,Y = symbols('X Y')

#Before I start off, this code will not make sense if you don't understand how classes and function definitions work.
#These comments are there for my sanity because I cannot finish this code in just one sitting.

#Current workflow:- Nodes -> Elements -> Stiffness Matrix -> ???? -> Solution.

class node: #Create a simple node, helps to keep track of what they are.
    def __init__(self, xCoord, yCoord):
        self.x = xCoord
        self.y = yCoord
        self.pos = [xCoord, yCoord]  

#The V2 architecture scheme uses arrays instead of letters schemes, which will make this more unreadable.
class element: #Create elements using nodes, triangles and quadilaterals both.
    def __init__(self, nodes = []):
        self.nodeList = [nodes[0], nodes[1], nodes[2]]
        try:
            self.nodeList.append(nodes[3])
        except:
            None
        self.stiffness = None
        self.shape = None
        
    @property
    def isTri(self): #Might help in futre debugging or execution? Idk.
        try: 
            if self.nodeList[3] == None: return False 
        except: return True

def setUpNodes(): #Sets up the node system. Can I make this visual? How spiteful do I feel?
    #n = input('How many nodes would you like to use?')
    x = [0,1,2,1,2,2]
    y = [0,0,0,1,1,2]
    nodes = []
    for i in range(len(x)): nodes.append(node(x[i],y[i]))
    return nodes
    
def setUpElements(nodes): #Sets up elements using nodes. Can I make this clockwise on its own? Most probably not.
    #e = input('How many elements would you like to use?')
    #'''
    e = 3
    elements = []
    connections = [[nodes[0],nodes[1],nodes[3]],
           [nodes[1],nodes[2],nodes[4],nodes[3]],
           [nodes[3],nodes[4],nodes[5]]]
    '''
    e = 4
    elements = []
    connections = [[nodes[0],nodes[1],nodes[3]],
           [nodes[1],nodes[2],nodes[4]],
           [nodes[1],nodes[4],nodes[3]],
           [nodes[3],nodes[4],nodes[5]]]
    '''
    for i in range(e): elements.append(element(connections[i]))
    return elements
    
def triStiff(elem): #Spits out stiffness matrix for any triangular element. This seems to work.
    a,b,c=  [zeros(3,1) for i in range(3)]
    x,y = [],[]
    for node in elem.nodeList:
        x.append(node.x)
        y.append(node.y)
    A = Matrix([[1,1,1], x, y]).det() / 2
    for i in range(3):              
        b[i] = (y[(i+1)%3] - y[(i+2)%3])                    
        c[i] = (x[(i+2)%3] - x[(i+1)%3])                    
    k = Matrix(3,3, lambda i,j: b[i]*b[j] + c[i]*c[j]) / (4*A)
    return k

def triShape(elem): #Spits out shape matrix for any triangular element. This seems to work.
    a,b,c=  [zeros(3,1) for i in range(3)]
    N = zeros(3,3)
    x,y = [],[]
    for node in elem.nodeList:
        x.append(node.x)
        y.append(node.y)
    A = Matrix([[1,1,1], x, y]).det() / 2
    for i in range(3):              #This loop automatically makes all the values, which can scale over to different codes.
        a[i] = (x[(i+1)%3]*y[(i+2)%3] - x[(i+2)%3]*y[(i+1)%3])
        b[i] = (y[(i+1)%3] - y[(i+2)%3])                   
        c[i] = (x[(i+2)%3] - x[(i+1)%3])                    
        N[i] = [a[i], b[i]*X, c[i]*Y]
    return N.T

def rectStiff(elem): #Spits out stiffness matrix for any rectangular element. This does not work.
    a,b,c=  [zeros(4,1) for i in range(3)]
    x,y = [],[]
    for node in elem.nodeList:
        x.append(node.x)
        y.append(node.y)
    A = 0.5 * ((x[0]*y[1] +x[1]*y[2] +x[2]*y[3] +x[3]*y[0]) - (x[1]*y[0] +x[2]*y[1] +x[3]*y[2] +x[0]*y[3]))
    for i in range(4):
        a[i] = (x[(i+1)%4]*y[(i+2)%4] - x[(i+2)%4]*y[(i+1)%4])
        b[i] = (y[(i+1)%4] - y[(i+2)%4])                    
        c[i] = (x[(i+2)%4] - x[(i+1)%4])                    
    k = Matrix(4,4, lambda i,j: a[i]*a[j] + b[i]*b[j] + c[i]*c[j]) / (6*A)
    return k

def rectShape(elem): #Spits out shape matrix for any rectangular element. This seems to work.
    a,b,c=  [zeros(4,1) for i in range(3)]
    N = zeros(4,4)
    x,y = [],[]
    for node in elem.nodeList:
        x.append(node.x)
        y.append(node.y)
    a = (x[1] - x[0]) / 2
    b = (y[2] - y[0]) / 2
    A = 4*a*b
    V = Matrix([1,X,Y,X*Y])
    C = Matrix([[1,0,0,0],
              [-1/(2*a),1/(2*a),0,0],
              [-1/(2*b),0,0,1/(2*b)],
              [1/(4*a*b),-1/(4*a*b),1/(4*a*b),-1/(4*a*b)]])
    N =  Matrix(4,4, lambda i,j: C[i,j]*V[i])
    return N.T

def globalMatrix(elements, nodes): #Returns the global stiffness matrix using the elements and nodes.
    n = len(nodes)
    Kg = zeros(n,n)
    for element in elements: #This was some smart mapping here. But I don't really remember what.
        nodePos = []
        for node in element.nodeList: nodePos.append(nodes.index(node))
        for i in nodePos:
            for j in nodePos:
                Kg[i,j] += element.stiffness[nodePos.index(i),nodePos.index(j)]
    return Kg
                
        
nodes = setUpNodes()
elements = setUpElements(nodes)
for element in elements: 
    if element.isTri == True: 
        element.stiffness = triStiff(element)
        element.shape = triShape(element)
    else: 
        element.stiffness = rectStiff(element)
        element.shape = rectShape(element)
    print('Element: ', elements.index(element)+1)
    display('Shape Matrix:',element.shape, 'Stiffness Matrix:',element.stiffness)
    
Kg = globalMatrix(elements, nodes)
display('Global Stiffness Matrix:',Kg)

Element:  1


'Shape Matrix:'

⎡1  -X  0 ⎤
⎢         ⎥
⎢0  X   -Y⎥
⎢         ⎥
⎣0  0   Y ⎦

'Stiffness Matrix:'

⎡1/2   -1/2   0  ⎤
⎢                ⎥
⎢-1/2   1    -1/2⎥
⎢                ⎥
⎣ 0    -1/2  1/2 ⎦

...

⎡2   -X  0 ⎤
⎢          ⎥
⎢0   X   -Y⎥
⎢          ⎥
⎣-1  0   Y ⎦

'Stiffness Matrix:'

⎡1/2   -1/2   0  ⎤
⎢                ⎥
⎢-1/2   1    -1/2⎥
⎢                ⎥
⎣ 0    -1/2  1/2 ⎦

'Global Stiffness Matrix:'

⎡1/2         -1/2                 0                   0                   0   
⎢                                                                             
⎢-1/2  1.83333333333333   0.333333333333333          -1/2                -0.5 
⎢                                                                             
⎢ 0    0.333333333333333  0.333333333333333   -0.166666666666667  -0.166666666
⎢                                                                             
⎢ 0          -1/2         -0.166666666666667   1.16666666666667          -1/2 
⎢                                                                             
⎢ 0          -0.5         -0.166666666666667         -1/2          1.333333333
⎢                                                                             
⎣ 0            0                  0                   0                  -1/2 

         0  ⎤
            ⎥
         0  ⎥
            ⎥
666667   0  ⎥
            ⎥
         0  ⎥
            ⎥
33333   -1/2⎥
    